<a href="https://colab.research.google.com/github/az2611/BerkeleyML_Module_11/blob/main/prompt_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The data consists of 17 campaigns that took place between May 2008 and November 2010, comprising a total of 79,354 contacts. These campaigns involved offering an attractive long-term deposit application with competitive interest rates. For each contact, a wide range of attributes were recorded, as well as whether the contact was successful (the target variable). In the overall dataset, there were 6,499 successful contacts (8% success rate).

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [72]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_curve, roc_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.svm import SVC

###
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
set_config(display="diagram")

set_config("figure")
import warnings
warnings.filterwarnings('ignore')

In [73]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
#df = pd.read_csv('data/bank-additional-full.csv', sep = ';')
df =pd.read_csv('/content/drive/My Drive/UC_Berkeley_Certificate/Module_17/module_17_starter/data/bank-additional-full.csv', sep = ';')

In [75]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



-Duration should be dropped from the predictive model as we do not know duration until the call is finished.  
-pdays needs to be coercd to categorial as not being prviously contacted can create issues

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The goal is to develop a model that can predict the outcome of a contact, specifically whether a client will subscribe to a deposit. This model will improve campaign efficiency by identifying key characteristics that affect success, enabling better management of resources such as human effort, phone calls, and time, as well as the selection of high-quality, cost-effective potential customer; resulting in increased efficiency of directed campaigns for long-term deposit subscriptions by reducing the number of contacts to do.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [77]:
df_clean = df.dropna() #To eliminate all rows with empty values
print(len(df))
print(len(df_clean))

41188
41188


In [78]:
X=df_clean.iloc[:, 0:7]
X.head()

,age,job,marital,education,default,housing,loan
0,56,housemaid,married,basic.4y,no,no,no
1,57,services,married,high.school,unknown,no,no
2,37,services,married,high.school,no,yes,no
3,40,admin.,married,basic.6y,no,no,no
4,56,services,married,high.school,no,no,yes


In [79]:
y=df_clean.y
y.head()

0    no
1    no
2    no
3    no
4    no
Name: y, dtype: object

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = df_clean.y, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [81]:
# Create an instance of the DummyClassifier
clf = DummyClassifier(strategy='most_frequent')

In [82]:
# Fit the classifier to the training data

import time

start = time.time()

clf.fit(X_train, y_train)

stop = time.time()
time = stop - start
print(f"Training time: {stop - start}s")

Training time: 0.01810598373413086s


In [83]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [84]:
# Calculate the accuracy of the classifier
baseline_test_accuracy = accuracy_score(y_test, clf.predict(X_test))
baseline_train_accuracy = accuracy_score(y_train, clf.predict(X_train))

print('baseline_test_accuracy:',baseline_test_accuracy)
print('baseline_train_accuracy:',baseline_train_accuracy)

baseline_test_accuracy: 0.8873458288821987
baseline_train_accuracy: 0.8873458288821987


In [85]:
# create a dictionary with the mean of each key-value pair
baseline_results = {'model': 'Baseline','Train Time': time, 'Train Accuracy': baseline_train_accuracy, 'Test Accuracy': baseline_test_accuracy}
baseline_results_df = pd.DataFrame(baseline_results, index=[0])
baseline_results_df

,model,Train Time,Train Accuracy,Test Accuracy
0,Baseline,0.018106,0.887346,0.887346


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [86]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), 
                                       ['job','marital','education','default','housing','loan',]),
                                      remainder = StandardScaler())

In [87]:
lgr_pipe = Pipeline([('transformer', transformer),('lr', LogisticRegression())])

In [88]:
import time

start = time.time()

lgr_pipe.fit(X_train, y_train)

stop = time.time()
time = stop - start
print(f"Training time: {stop - start}s")

Training time: 0.4761533737182617s


#### Problem 9: Score the Model

What is the accuracy of your model?

In [89]:
lgr_test_accuracy = accuracy_score(y_test, lgr_pipe.predict(X_test))
lgr_train_accuracy = accuracy_score(y_train, lgr_pipe.predict(X_train))

print('lgr_test_accuracy:',lgr_test_accuracy)
print('lgr_train_accuracy:',lgr_train_accuracy)

lgr_test_accuracy: 0.8873458288821987
lgr_train_accuracy: 0.8873458288821987


In [90]:
# create a dictionary with the mean of each key-value pair
lgr_results = {'model': 'Logistic Regression','Train Time': time, 'Train Accuracy': lgr_train_accuracy, 'Test Accuracy': lgr_test_accuracy}
lgr_results_df = pd.DataFrame(lgr_results, index=[0])
lgr_results_df

,model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.476153,0.887346,0.887346


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

#### KNN algorithm

In [91]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['job','marital','education','default','housing','loan',]),
                                     remainder = StandardScaler())

In [92]:
knn_pipe = Pipeline([('transformer', transformer),('knn', KNeighborsClassifier())])

In [93]:
import time

start = time.time()

knn_pipe.fit(X_train, y_train)

stop = time.time()
time_knn = stop - start
print(f"Training time: {stop - start}s")

Training time: 0.13619256019592285s


In [94]:
knn_test_accuracy = accuracy_score(y_test, knn_pipe.predict(X_test))
knn_train_accuracy = accuracy_score(y_train, knn_pipe.predict(X_train))

print('knn_test_accuracy:',knn_test_accuracy)
print('knn_train_accuracy:',knn_train_accuracy)

knn_test_accuracy: 0.8772457997474993
knn_train_accuracy: 0.8910685960312065


In [95]:
# create a dictionary with the mean of each key-value pair
knn_results = {'model': 'KNN','Train Time': time_knn, 'Train Accuracy': knn_train_accuracy, 'Test Accuracy': knn_test_accuracy}
knn_results_df = pd.DataFrame(knn_results, index=[0])
knn_results_df

,model,Train Time,Train Accuracy,Test Accuracy
0,KNN,0.136193,0.891069,0.877246


#### Decision Tree

In [96]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['job','marital','education','default','housing','loan',]),
                                     remainder = StandardScaler())

In [97]:
DT_pipe = Pipeline([('transformer', transformer),('DT', DecisionTreeClassifier())])

In [98]:
import time
start = time.time()

DT_pipe.fit(X_train, y_train)

stop = time.time()
time = stop - start
print(f"Training time: {stop - start}s")

Training time: 0.8143112659454346s


In [99]:
DT_test_accuracy = accuracy_score(y_test, DT_pipe.predict(X_test))
DT_train_accuracy = accuracy_score(y_train, DT_pipe.predict(X_train))

print('DT_train_accuracy:',DT_train_accuracy)
print('DT_test_accuracy:',DT_test_accuracy)

DT_train_accuracy: 0.9177754038393059
DT_test_accuracy: 0.8668544236185297


In [100]:
# create a dictionary with the mean of each key-value pair
DT_results = {'model': 'Decision Tree','Train Time': time, 'Train Accuracy': DT_train_accuracy, 'Test Accuracy': DT_test_accuracy}
DT_results_df = pd.DataFrame(DT_results, index=[0])
DT_results_df

,model,Train Time,Train Accuracy,Test Accuracy
0,Decision Tree,0.814311,0.917775,0.866854


#### SVM

In [101]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['job','marital','education','default','housing','loan',]),
                                     remainder = StandardScaler())

In [102]:
SVC_pipe = Pipeline([('transformer', transformer),('svc', SVC())])

In [103]:
import time
start = time.time()

SVC_pipe.fit(X_train, y_train)

stop = time.time()
time = stop - start
print(f"Training time: {stop - start}s")

Training time: 88.07465100288391s


In [104]:
SVC_test_accuracy = accuracy_score(y_test, SVC_pipe.predict(X_test))
SVC_train_accuracy = accuracy_score(y_train, SVC_pipe.predict(X_train))

print('SVC_train_accuracy:',SVC_train_accuracy)
print('SVC_test_accuracy:',SVC_test_accuracy)

SVC_train_accuracy: 0.8880903823120002
SVC_test_accuracy: 0.8864717878993882


In [105]:
SVC_results = {'model': 'SVM','Train Time': time, 'Train Accuracy': SVC_train_accuracy, 'Test Accuracy': SVC_test_accuracy}
SVC_results_df = pd.DataFrame(SVC_results, index=[0])
SVC_results_df

,model,Train Time,Train Accuracy,Test Accuracy
0,SVM,88.074651,0.88809,0.886472


#### Findings

In [106]:
final_results = pd.concat([baseline_results_df, lgr_results_df, knn_results_df, DT_results_df, SVC_results_df ], axis=0)
final_results = final_results.sort_values(by='Test Accuracy', ascending=False)
final_results

,model,Train Time,Train Accuracy,Test Accuracy
0,Baseline,0.018106,0.887346,0.887346
0,Logistic Regression,0.476153,0.887346,0.887346
0,SVM,88.074651,0.888090,0.886472
0,KNN,0.136193,0.891069,0.877246
0,Decision Tree,0.814311,0.917775,0.866854


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

#### Logistic Regression

In [107]:
lgr_params = {'lr__C': [0.1, 1, 10, 100, 1000],
              'lr__penalty': ['l1', 'l2'],
              'lr__solver': ['newton-cg', 'lbfgs', 'liblinear']}

In [108]:
logreg_cv=GridSearchCV(lgr_pipe, param_grid=lgr_params, cv=3, verbose=1)

In [109]:
import time
start = time.time()

logreg_cv.fit(X_train,y_train)

stop = time.time()
print(f"Training time: {stop - start}s")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Training time: 31.121613264083862s


In [110]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
accuracy : 0.8873458288821987


In [111]:
mean_fit_time= logreg_cv.cv_results_['mean_fit_time']
mean_score_time= logreg_cv.cv_results_['mean_score_time']
n_splits  = logreg_cv.n_splits_ #number of splits of training data
n_iter = pd.DataFrame(logreg_cv.cv_results_).shape[0] #Iterations per split

In [112]:
CV_lgr_results = {'model': 'GridSearch Logistic Regression','Accuracy': logreg_cv.best_score_, 'Time':[np.mean(mean_fit_time + mean_score_time) * n_splits * n_iter]}
CV_lgr_results = pd.DataFrame(CV_lgr_results, index=[0])
CV_lgr_results

,model,Accuracy,Time
0,GridSearch Logistic Regression,0.887346,30.361033


#### KNN algorithm

In [113]:
knn_params = {'knn__n_neighbors': range(1,50,5),
          'knn__weights': ['uniform', 'distance'],
          'knn__p': [1,2]}

In [114]:
knn_grid = GridSearchCV(knn_pipe, param_grid = knn_params, cv=3,verbose=1)

In [115]:
import time
start = time.time()

knn_grid.fit(X_train,y_train)

stop = time.time()
print(f"Training time: {stop - start}s")

Fitting 3 folds for each of 40 candidates, totalling 120 fits
Training time: 1650.3121683597565s


In [116]:
print("tuned hpyerparameters :(best parameters) ",knn_grid.best_params_)
print("accuracy :",knn_grid.best_score_)

tuned hpyerparameters :(best parameters)  {'knn__n_neighbors': 36, 'knn__p': 2, 'knn__weights': 'uniform'}
accuracy : 0.8872163413291897


In [117]:
mean_fit_time= knn_grid.cv_results_['mean_fit_time']
mean_score_time= knn_grid.cv_results_['mean_score_time']
n_splits  = knn_grid.n_splits_ #number of splits of training data
n_iter = pd.DataFrame(knn_grid.cv_results_).shape[0] #Iterations per split

In [118]:
CV_knn_results = {'model': 'GridSearch KNN','Accuracy': knn_grid.best_score_, 'Time':[np.mean(mean_fit_time + mean_score_time) * n_splits * n_iter]}
CV_knn_results = pd.DataFrame(CV_knn_results, index=[0])
CV_knn_results

,model,Accuracy,Time
0,GridSearch KNN,0.887216,1649.781925


#### Decision Tree

In [119]:
DT_params = {'DT__max_depth': [4,5,6,7,8,9,10],
             'DT__min_samples_split': [2,4,6,8],
             'DT__criterion': ['gini','entropy'],
             'DT__min_samples_leaf': [1,2]}

In [120]:
gscv = GridSearchCV(DT_pipe, param_grid=DT_params, cv=3, verbose=1)

In [121]:
import time
start = time.time()

gscv.fit(X_train,y_train)

stop = time.time()
print(f"Training time: {stop - start}s")

Fitting 3 folds for each of 112 candidates, totalling 336 fits
Training time: 66.0078477859497s


In [122]:
print("tuned hpyerparameters :(best parameters) ",gscv.best_params_)
print("accuracy :",gscv.best_score_)

tuned hpyerparameters :(best parameters)  {'DT__criterion': 'gini', 'DT__max_depth': 7, 'DT__min_samples_leaf': 2, 'DT__min_samples_split': 8}
accuracy : 0.8864717878993882


In [123]:
mean_fit_time= gscv.cv_results_['mean_fit_time']
mean_score_time= gscv.cv_results_['mean_score_time']
n_splits  = gscv.n_splits_ #number of splits of training data
n_iter = pd.DataFrame(gscv.cv_results_).shape[0] #Iterations per split

In [124]:
CV_DT_results = {'model': 'GridSearch Decision Tree','Accuracy': gscv.best_score_, 'Time':[np.mean(mean_fit_time + mean_score_time) * n_splits * n_iter]}
CV_DT_results = pd.DataFrame(CV_DT_results, index=[0])
CV_DT_results

,model,Accuracy,Time
0,GridSearch Decision Tree,0.886472,65.091757


#### SVM

In [125]:
svc_params = {'svc__kernel': ['linear', 'sigmoid'],
              'svc__C': [0.001, 0.1, 1],
              'svc__gamma': [1, 0.1, 0.01]}

In [126]:
SVC_grid = GridSearchCV(SVC_pipe, param_grid=svc_params, cv=3, verbose=1)

In [127]:
import time
start = time.time()

SVC_grid.fit(X_train,y_train)

stop = time.time()
print(f"Training time: {stop - start}s")

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Training time: 611.9505178928375s


In [128]:
print("tuned hpyerparameters :(best parameters) ",SVC_grid.best_params_)
print("accuracy :",SVC_grid.best_score_)

tuned hpyerparameters :(best parameters)  {'svc__C': 0.001, 'svc__gamma': 1, 'svc__kernel': 'linear'}
accuracy : 0.8873458288821987


In [129]:
mean_fit_time= SVC_grid.cv_results_['mean_fit_time']
mean_score_time= SVC_grid.cv_results_['mean_score_time']
n_splits  = SVC_grid.n_splits_ #number of splits of training data
n_iter = pd.DataFrame(SVC_grid.cv_results_).shape[0] #Iterations per split

In [130]:
CV_SVC_results = {'model': 'GridSearch Decision Tree','Accuracy': SVC_grid.best_score_, 'Time':[np.mean(mean_fit_time + mean_score_time) * n_splits * n_iter]}
CV_SVC_results = pd.DataFrame(CV_SVC_results, index=[0])
CV_SVC_results

,model,Accuracy,Time
0,GridSearch Decision Tree,0.887346,599.879015


#### GridSearchCV Results

In [131]:
gs_final_results = pd.concat([CV_lgr_results,CV_knn_results,CV_DT_results,CV_SVC_results], axis=0)


In [132]:
gs_final_results = gs_final_results.sort_values(by='Accuracy', ascending=False)
gs_final_results

,model,Accuracy,Time
0,GridSearch Logistic Regression,0.887346,30.361033
0,GridSearch Decision Tree,0.887346,599.879015
0,GridSearch KNN,0.887216,1649.781925
0,GridSearch Decision Tree,0.886472,65.091757
